# Transformer疑问以及补充笔记

## 1、Transformer整体架构

### 1.3、一句话是如何变成向量的：

1、分词：将句子分成一个个词汇   
2、嵌入特殊标记：开头加入"`<sos>`"，末尾加入"`<eos>`" ，特殊标记也会被视作一个词。  
3、数值化：对照自己构建的或用别人预训练的词汇表进行数值化，数值化后通常是一个整数列表  
4、嵌入层转换：使用嵌入层将每一个ID转换为密集向量，并进行**缩放**（进行幅度匹配，避免导致后续相加的位置编码被无效化）输出三维张量(batch_size, seq_len, d_model)。    
&ensp;&ensp;嵌入层：一个强化版的词汇表，不仅记录有哪些词，还为这些词分配了一个**语义向量（推荐512维，性价比最高）**。  
5、位置编码：为每一个词添加位置信息，transformer没有位置的概念，不添加位置信息，打乱句子顺序后的输出是一样的。一个词向量如果有512维，他的位置信息也有512维。10000是缩放常数，是用来控制正弦波和余弦波的频率范围。对于偶数维度使用sin位置编码，对于奇数维度使用cos位置编码。  
    
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$  
6、嵌入与位置编码相加：将词汇表中每个词查表得到的 512维词嵌入向量，与对应位置通过公式计算出的 512维位置编码向量，进行向量加法，得到最终输入到 Transformer 的表示。  


关于一个问题的解释：既然位置信息都已经加到了词向量中，模型又怎么得知位置的信息呢？  
相加是一种优雅的隐式编码：它让位置信息**渗透**到每个维度，而不是作为一个外部信号。当一个词出现在不同位置时，它的**混合向量**会不同，导致它在上下文中的行为也不同。



## 2、Transformer的q,k,v过程

### 2.2、q,k,v过程

q代表查询，k代表键向量，与查询向量匹配，计算注意力分数，v代表值向量，代表实际内容。  
通过嵌入层得到的向量表示x，并设置权重矩阵Wk,Wq,Wv，这三个矩阵维度是由嵌入维度和注意力头数决定。Wk,Wq,Wv的维度都是 d_model × d_model，但它们**内部**被分成了 num_heads 份，每份对应一个头，大小为 d_model × (d_model // num_heads)。  

**计算q,k,v：**

以x中的x1（第一个词的嵌入维度）举例：  
$$ 
\begin{aligned}
Q_1 = x_1 \cdot Wq \\  
K_1 = x_1 \cdot Wk \\ 
V_1 = x_1 \cdot Wv  
\end{aligned}
$$
这些就是x1的q,k,v

**计算注意力分数:**

需要计算每一个词之间的注意力分数，包括和自己。  
$$
\begin{aligned}
score(Q_1,K_1)=Q_1 \cdot K_1^\top \\
score(Q_1,K_2)=Q_1 \cdot K_2^\top \\
score(Q_1,K_3)=Q_1 \cdot K_3^\top 
\end{aligned}
$$



**进行缩放:**

先明确一个值d_K:$$d_k = \frac{d_{\text{model}}}{\text{num\_heads}}$$  
num_heads是注意力头数量。  
缩放公式（**以第一个查询位置对自己的注意力分数为例，后面都是这样**）：$$\text{scaled\_score}(Q_1,K_1)=\frac{score(Q_1,K_1)}{\sqrt{d_k}}$$  
缩放将对**整个注意力分数矩阵**进行缩放，注意力分数矩阵的每一行代表着一个“查询位置”（Query Position）对所有“键位置”（Key Positions）的注意力分数。

**进行掩码操作**：

*见后续第五节*

**进行softmax操作:**

对所有缩放后的分数应用Softmax函数，得到**注意力权重**。  
$$
\text{attention\_weights}=softmax(\text{scaled\_score}(Q_1,K_1))$$  
这个同样也是对**整个注意力分数矩阵**进行操作。

**计算注意力输出：**

将x1的注意力权重与x1的V向量相乘，得到注意力输出这个过程对每个词都会执行一次，最终得到整个序列的注意力输出（单个头，如果是多头还要再concat）

**上述操作的流程图总结：**

此处假定序列x,有n个词，注意力多头设置的为m,d_model设为512维

<img src="../image/Q-K-V流程图.png" alt="Q-K-V流程图" style="width:auto; height:auto;">

**上述操作的代码流程：**

In [2]:
import torch
import torch.nn as nn

def softmax(x):
    return torch.softmax(x,dim=-1) #对最后一个维度进行操作，此处为列方向，对列进行压缩（归一化），也就是对每一个查询的注意力分数向量进行归一化操作

def attention_score(Q, K, scale=True, d_k=None):
    scores = torch.matmul(Q, K.T)
    if scale and d_k is not None:
        scores = scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
    return scores

#假设的值，序列假设为“我/爱/学习”
Q1 = torch.tensor([0.7, 0.8, 0.9])   # "我"的查询向量
K = torch.tensor([[0.9, 1.0, 1.1],    # "我"的键向量
                  [1.3, 1.4, 1.5],    # "爱"的键向量  
                  [1.7, 1.8, 1.9]])   # "学习"的键向量
V = torch.tensor([[1.1, 1.2, 1.3],    # "我"的值向量
                  [1.5, 1.6, 1.7],    # "爱"的值向量
                  [1.9, 2.0, 2.1]])   # "学习"的值向量

d_k = 3 #键向量维度

#计算注意力分数
scores = attention_score(Q1, K, scale=False)
print(f"原始注意力分数: {scores}")
# 缩放处理
scaled_scores = attention_score(Q1, K, scale=True, d_k=d_k)
print(f"缩放后注意力分数: {scaled_scores}")
#softmax归一化
attention_weights = softmax(scaled_scores)
print(f"注意力权重: {attention_weights}")
#计算注意力输出
output = torch.matmul(attention_weights, V)
print(f"注意力输出: {output}")
# 对比实验：缩放 vs 不缩放
print("\n=== 缩放效果对比 ===")
weights_no_scale = softmax(scores)
weights_with_scale = softmax(scaled_scores)
print(f"不缩放时的权重分布: {weights_no_scale}")
print(f"缩放后的权重分布:  {weights_with_scale}")
print(f"缩放让分布更均匀，避免极端值!")

原始注意力分数: tensor([2.4200, 3.3800, 4.3400])
缩放后注意力分数: tensor([1.3972, 1.9514, 2.5057])
注意力权重: tensor([0.1733, 0.3016, 0.5251])
注意力输出: tensor([1.6407, 1.7407, 1.8407])

=== 缩放效果对比 ===
不缩放时的权重分布: tensor([0.0959, 0.2503, 0.6538])
缩放后的权重分布:  tensor([0.1733, 0.3016, 0.5251])
缩放让分布更均匀，避免极端值!


# 3、残差连接（Residual Connection）

### 残差连接（跳跃连接）的基础概念：

残差连接是⼀种网络结构设计，它允许信息绕过某些层直接向前传播。在Transformer中，残差连接将子层的输⼊直接加到子层的输出上。
$$Output = x + Sublayer(x)$$  

#### 核心优势：  
**信息不丢失**：原始输入 x 的信息不会完全丢失，它会以 x + ... 的形式保留下来，**相当于是在原始信息上添加修正而不是重写信息。**  
**解决深层网络梯度消失的问题**：为梯度提供捷径，让梯度可以直接从深层流回浅层，而不需要经过所有中间层的非线性变换。**确保梯度高效传递，浅层也能顺利更新参数**从而加速模型收敛。    
**训练更稳定**：残差连接使得网络的输出变化更加平滑。即使子层 F(x) 的输出发生剧烈变化，最终输出 x + F(x) 也不会完全偏离原始输入。    
**促进深层网络训练**：残差连接使得训练非常深的网络成为可能。  
**全覆盖**：每个子层都有残差连接。  
**协同设计**：隐含“恒等映射”先验，使网络从稳定状态开始学习。**[“恒等映射”先验:残差连接隐含假设：最优变换 F(x) 接近于 0。在训练初期，权重随机初始化，F(x) 通常很小，网络行为接近 h ≈ x。这是一个合理的、稳定的起点，比一个完全随机的非线性变换更容易优化]**

#### 弱点补充：  
残差连接防止信息丢失但不保证数值稳定，因此必须与层归一化连用，让层归一化来稳定激活值（本层的输出，下层的输入）的分布。  
$$ Output=LayerNorm(x+Sublayer(x))$$  
这种结合的好处是：  
1.稳定性：层归一化有助于稳定每一层的输入分布，使得训练更加稳定。  
2.收敛速度：稳定的训练可以避免**损失函数剧烈波动**，可以加速模型的收敛过程。  
3.正则化效果：层归⼀化具有⼀定的正则化效果，有助于防⽌过拟合  

### 层归一化详解:

对残差连接的结果进行层归一化。层归一化的计算公式为：  
$$
LayerNorm(x)=\gamma \cdot \frac{x-\mu}{\sqrt{\sigma^2+\epsilon}}+\beta
$$  
其中：  
μ是均值   
σ^2是方差，sqrt(σ^2)是标准差std  
方差计算公式：  
（有偏）$$\sigma^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2$$  
（无偏）$$\sigma^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \mu)^2$$  
γ,β和是可学习的参数  
ϵ是⼀个很⼩的常数，防止除以零  

# 4、FFN前馈神经网络

**FFN的作用：**  
1、非线性变换：为模型引入非线性表达能力  
2、特征提取：从注意力机制的输出中提取更高级的特征  
3、维度变换，在不同维度的空间之间进行映射，增强模型的表达能力  

**FFN的作用范围：**  
FFN是**逐位置**应用的，它将独立的作用于序列中的每一个token：  
第 1 个 token的表示只通过 FFN 计算得到新的第 1 个 token表示    
第 2 个 token的表示只通过 FFN 计算得到新的第 2 个 token表示   
它们不共享计算路径，但共享参数（即所有 token 使用同一组 W1, b1, W2, b2）  
假设输入是一个 (n, 512) 的矩阵，FFN 实际上是对这个矩阵的每一行（每个 token 的 512 维向量）应用同一个两层神经网络。

**FFN的结构：**  
FFN通常由两个线性变换（全连接层）和一个relu激活函数组成：  
$$
FFN(x) = max(0,x\cdot W_1+b_1)\cdot W_2+b_2
$$  
隐藏层维度d_ff通常设置为：2048。    
w_1需要将输入拓展至隐藏层，维度为(512,2048)，此时的输出维度为（n,2048)。    
同时，偏置b_1维度为（2048,），偏置是一个可学习的参数，它的大小只取决于它所在层的输出特征数，在计算时，框架会自动**进行广播**，将b_1复制n份，形成维度为（n,2048）的矩阵。      
应用relu激活函数。    
w_2需要将输出压缩回d_model,维度为（2048,512),此时输出维度为（n,512），同时，偏置b_2（512，）原理同b_1。  

#### FNN计算过程总结：

线性变换（升维），relu激活函数，线性变换（降维），残差连接，层归一化

**一些问题的解释：**  

1、既然都是序列X整个传入，整个与权重矩阵w_1相乘，再整个与偏置矩阵b_1相加，那么有什么必要说是作用于单个token呢？  
这是**实现方式**（向量化批量处理）和**概念本质**（逐位置应用）之间的表面矛盾。整个传入是为了效率，pytorch的矩阵运算是高效的，其向量化实现是对n个独立计算的并行化，没有引入token之间的任何交互（可以对比自注意力机制）。  
2、维度扩展的目的：
FFN首先将输入从维度扩展到维度（通常是的4倍），这样做的主要目的是：  
**增加表示能力**：高维空间提供了更多的表示能力，可以捕获更复杂的特征和模式。   
**特征解耦**：在高维空间中，不同特征更容易被解耦，使得模型能够学习到更丰富的表示。   
**信息重组**：通过扩展维度，模型可以对输⼊信息进行重组和重新组合，发现新的特征组合。  
**[可以看作是一个特征提取器，将原始特征线性组合成新的高维特征]**   
3、维度压缩的目的：  
在扩展维度后，FFN又将表示从d_ff维度压缩回d_model维度，这样做的主要目的是：   
**减少显存消耗**：可以显著减少内存占用，使得模型能够在有限的GPU显存下处理更长的序列或更大的批次  
**信息筛选**：压缩过程可以看作是一种信息筛选，保留最重要的信息，丢弃不重要的信息，同时效果几乎保持不变  
**维度匹配**：压缩后的维度与输入维度相同，便于残差连接和层间的信息传递  
**防止过拟合**：通过压缩维度，可以减少模   
**[可以看作一个特征选择器，从高维特征中选取最重要的特征压缩回原维度]**    
4、为什么FNN只用做一个残差连接。而不是在每次线性变换后都做？  
FFN 本身不是“深层”网络：它只有两层，梯度消失问题不严重。  
设计哲学是“模块化”：残差连接作用于子层（Sublayer），而不是子层内部。  
避免破坏 FFN 的功能：FFN 的核心是“升维 → 非线性 → 降维”的信息重组，内部加残差会削弱其表达能力。  
参数效率与简洁性：一次残差连接已足够，无需复杂化。  

**FFN与子层连接代码示例：**

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForwardNetwork(nn.Module):
    """
    标准的前馈神经网络（FFN）模块，用于 Transformer。 
    结构：Linear(d_model -> d_ff) -> ReLU -> Linear(d_ff -> d_model)
    """
    def __init__(self, d_model=512, d_ff=2048, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)#第一次线性变换
        self.linear2 = nn.Linear(d_ff, d_model)#第二次线性变换
        self.dropout = nn.Dropout(dropout)
        #self._init_weights()#初始化该方法后使用的就是自定义的初始化，不添加默认使用kaiming初始化
        
    #好的初始化能让网络在训练初期就处于一个“良好状态”，加速收敛
    def _init_weights(self):
        """自定义权重初始化，PyTorch 的 nn.Linear 默认使用 Kaiming 初始化（适合 ReLU）"""
        nn.init.xavier_uniform_(self.linear1.weight)#Xavier初始化，适合tanh 或 sigmoid激活函数
        nn.init.xavier_uniform_(self.linear2.weight)
        nn.init.constant_(self.linear1.bias, 0.0)# 初始化偏置为0
        nn.init.constant_(self.linear2.bias, 0.0)

    #前向传播
    def forward(self, x):
        """
        参数:
            x: 输入张量，形状 (batch_size, seq_len, d_model) 或 (d_model,)
            nn.Linear 自动处理 batch 和 seq 维度，只对最后一个维度做线性变换        
        返回:
            输出张量，形状与输入相同
        """
        h = self.linear1(x)           # 扩展到高维
        a = F.relu(h)                 # 非线性激活
        dropped = self.dropout(a)     # Dropout（可选，增强泛化）
        y = self.linear2(dropped)     # 压缩回原维度
        return y


#  子层连接（Sublayer Connection）
class SublayerConnection(nn.Module):
    """
    残差连接 + 层归一化，支持 Pre-LN 和 Post-LN。
    要求：sublayer(x) 的输出形状必须与 x 相同。
    """ 
    def __init__(self, d_model, dropout=0.1, pre_ln=False):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)#层归一化
        self.dropout = nn.Dropout(dropout)#dropout正则化
        self.pre_ln = pre_ln#选择模式
    
    def forward(self, x, sublayer):
        if self.pre_ln:
            # Pre-LN: 先归一化，子层处理，再残差连接
            sublayer_output = sublayer(self.norm(x))#层归一化
            return x + self.dropout(sublayer_output)#残差连接（dropout正则化）
        else:
            # Post-LN（原始）: 子层处理，残差连接，最后归一化
            sublayer_output = sublayer(x)
            return self.norm(x + self.dropout(sublayer_output))

#封装一个 工厂函数，一次直接生成 FFN 和 Sublayer
def create_standard_ffn_block(d_model=4, d_ff=8, pre_ln=False, dropout=0.1):
    """
    创建一个标准的 FFN 块（包含 FFN + SublayerConnection）
    使用 FeedForwardNetwork 内部的 _init_weights() 进行自动初始化，
    """
    # 创建 FFN 实例
    ff_network = FeedForwardNetwork(d_model, d_ff, dropout=dropout)
    
    # 使用类内部定义的 _init_weights() 方法进行自动初始化
    ff_network._init_weights()
    
    # 创建子层连接器
    sublayer = SublayerConnection(d_model, dropout=dropout,pre_ln=pre_ln)
    
    return ff_network, sublayer

#演示函数   
def ffn_example():
    d_model = 4
    x = torch.tensor([[1.0, 2.0, 3.0, 4.0]])  # (1, 4) 批量维度
    
    print(f"输入 x = {x}")
    
    # 创建 FFN 模块和子层连接
    ff_network, sublayer = create_standard_ffn_block(d_model=4, d_ff=8, pre_ln=True)   

    print("\n--- 使用 SublayerConnection ---")
    
    output = sublayer(x, ff_network)
    print(f"最终输出 = {output}")
    
    return output


In [11]:
def ffn_dimension_analysis():
    """FFN维度变化分析"""
    print("\n=== FFN维度变化分析 ===")
    
    configs = [
        {"d_model": 512, "d_ff": 2048},
        {"d_model": 256, "d_ff": 1024},
        {"d_model": 128, "d_ff": 512},
        {"d_model": 64, "d_ff": 256},
    ]
    
    for config in configs:
        d_model = config["d_model"]
        d_ff = config["d_ff"]
        
        params_W1 = d_model * d_ff
        params_b1 = d_ff
        params_W2 = d_ff * d_model
        params_b2 = d_model
        
        total_params = params_W1 + params_b1 + params_W2 + params_b2
        ratio = d_ff / d_model
        
        print(f"d_model={d_model}, d_ff={d_ff}:")
        print(f"  参数量: W1={params_W1:>8,}, W2={params_W2:>8,}, 总计={total_params:>9,}")
        print(f"  扩展比例: {ratio:.1f}x")


In [14]:
if __name__ == "__main__":
    result = ffn_example()
    ffn_dimension_analysis()

输入 x = tensor([[1., 2., 3., 4.]])

--- 使用 SublayerConnection ---
最终输出 = tensor([[0.8159, 2.3109, 3.0000, 4.9190]], grad_fn=<AddBackward0>)

=== FFN维度变化分析 ===
d_model=512, d_ff=2048:
  参数量: W1=1,048,576, W2=1,048,576, 总计=2,099,712
  扩展比例: 4.0x
d_model=256, d_ff=1024:
  参数量: W1= 262,144, W2= 262,144, 总计=  525,568
  扩展比例: 4.0x
d_model=128, d_ff=512:
  参数量: W1=  65,536, W2=  65,536, 总计=  131,712
  扩展比例: 4.0x
d_model=64, d_ff=256:
  参数量: W1=  16,384, W2=  16,384, 总计=   33,088
  扩展比例: 4.0x


# 5、编码器与解码器

**编码器**由N个相同的层堆叠而成（在原始transformer架构中，编码器的层数是N=6）每层包含两个子层：  
1、多头自注意力机制（Multi-Head Self-Attention）  
2、前馈神经网络（Feed Forward Network）  
每个子层都有残差连接和层归⼀化（Layer Normalization）  

#### 编码器的工作流程图：

<img src="..\image\编码器工作流程.png"  alt="编码器流程图" style="width:auto; height:auto;">

**解码器**也由N个相同的层堆叠而成（在原始Transformer中，N=6），每个层包含三个子层：  
1、 掩码多头自注意力机制（Masked Multi-Head Self-Attention）  
2、编码器解码器注意力机制（Encoder-Decoder Attention）[交叉注意力机制]  
3、前馈神经网络（Feed Forward Network）  
同样，每个子层都有残差连接和层归归一化。  

**交叉注意力层：**

交叉注意力层的作用是：让解码器在生成每一个目标词时，能够有选择地“关注”编码器对输入序列的编码结果，从而获取源序列的语义信息。  
Q来自：来自本层的掩码自注意力输出  
K，V来自：来自编码器最后一层的输出  

交叉注意力提供了：  
源语言的语义上下文  
对齐信息（通过注意力权重）  
信息桥接：把编码器的“理解”传递给解码器的“生成”  

### 编码器与解码器的掩码机制：

掩码：用于控制注意力机制中哪些位置可以关注，哪些位置不能关注。掩码通常是⼀个⼆进制矩阵，其中1表示允许关注，0表示不允许关注。  
主要作用：  
1、防止关注到填充位置  
2、防止关注到未来位置  
3、控制信息流动方向

#### 编码器中的掩码：

**填充掩码**：  
作用：防止注意力机制关注到输⼊序列中的填充位置。  
原因：在处理变长序列时，通常会将所有序列填充到相同的⻓度，以便进行批量处理。填充位置不包含任何有意义的信息，因此应该被忽略。  
实现：填充掩码是⼀个与输入序列相同长度的二进制向量，其中填充位置对应0，非填充位置对应1  

在计算注意力分数时，填充掩码会被应用到注意力分数上，将填充位置的注意力分数设置为⼀个非常小的值（如负无穷），这样在应用Softmax后，这些位置的注意力权重就会接近于0

#### 解码器中的掩码：

**填充掩码**：  
同上  
**前瞻掩码**：  
作用：防止解码器在预测当前位置时关注到未来的位置。  
原因：在训练过程中，解码器需要根据已经生成的序列来预测下一个词，而不是根据整个序列。这模拟了实际推理过程中的情况，即解码器只能基于已经生成的词来预测下⼀个词。  
实现：前瞻掩码是⼀个上三角矩阵，其中对角线及其以下的元素为1，对角线以上的元素为0。  
**交叉注意力掩码**：  
在解码器的交叉注意力机制中，解码器关注编码器的输出。这里只使用编码器的填充掩码，不使用前瞻掩码。


在解码器的自注意力机制中，需要同时应用填充掩码和前瞻掩码。最终的掩码是两个掩码的**逐元素乘积**。

# 六、十道关于transformer核心的验证题

1. transformer整体架构是怎么样的，几个部分组成？
2. 介绍transformer，详细介绍QKV过程？
3. 讲⼀讲tansformer里的encoder和decoder，以及整体工作原理，还有交叉注意力
4. transformer计算过程，softmax为什么要进⾏缩放？
5. 解释前馈神经网络（FFN）在Transformer中的作用及其设计理念
6. 为什么FFN要将高维度映射回低维度去呢？
7. 什么是残差连接，解释残差连接（Residual Connection）在Transformer中的作用及必要性
8. FFN块的计算公式是什么？
9. encoder和decoder中掩码的区别
10. Transformer怎么处理文字输入的？以及Transformer的输出是什么？